In [1]:
import pandas as pd
import random

In [2]:
df = pd.read_csv("words.csv")
df[:5]

,word
0,阿鼻地狱
1,阿党比周
2,阿党相为
3,阿狗阿猫
4,阿姑阿翁


In [3]:
df['len'] = df['word'].apply(len)
df[:5]

,word,len
0,阿鼻地狱,4
1,阿党比周,4
2,阿党相为,4
3,阿狗阿猫,4
4,阿姑阿翁,4


In [4]:
len(df)

30895

In [5]:
cys = df[df['len'] <= 10].reset_index(drop=True)
cys[:5]

,word,len
0,阿鼻地狱,4
1,阿党比周,4
2,阿党相为,4
3,阿狗阿猫,4
4,阿姑阿翁,4


In [6]:
cys['len'].describe()

count    30835.000000
mean         4.115291
std          0.678317
min          3.000000
25%          4.000000
50%          4.000000
75%          4.000000
max         10.000000
Name: len, dtype: float64

In [7]:
#当前关卡成语个数
def getCyNum(lvl):
    return 5 + round(lvl**0.25)

#位置数组
def getInxs(nanNum,wordLen):
    nanInxs = []
    for i in range(0,nanNum):
        inx = random.randint(0,wordLen-1)
        if inx not in nanInxs:
            nanInxs.append(inx)
    nanInxs.sort()
    return nanInxs

In [8]:
# 查找成语
def searchCy(key_word,inx):
    word = ''
    if key_word:
        tmp_cys = cys[cys['word'].str[inx:inx+1] == key_word].reset_index(drop=True)
    else:
        tmp_cys = cys
    #取成语
    tmp_len = len(tmp_cys)
    if tmp_len > 0:
        tmp_inx = random.randint(0,tmp_len-1)
        word = tmp_cys.loc[tmp_inx,'word']
    return word

# 查找成语2
def searchCy2(key_word,inx,key_word2,inx2):
    word = ''
    if key_word:
        tmp_cys = cys[(cys['word'].str[inx:inx+1] == key_word) & (cys['word'].str[inx2:inx2+1] == key_word2)].reset_index(drop=True)
    else:
        tmp_cys = cys
    #取成语
    tmp_len = len(tmp_cys)
    if tmp_len > 0:
        tmp_inx = random.randint(0,tmp_len-1)
        word = tmp_cys.loc[tmp_inx,'word']
    return word

# 获取冲突关键字
def getKeyWord2(inx):
    key_word = ''
    for item in res_word_dots:
        if item.find(inx) != -1:
            key_word = item.replace(inx,'')
            break
    return key_word

In [9]:
#第一个
def buildCy(direct,dots):
    #确定方向
    for dot in dots:
        print("=====坐标起点1: ",dot)
        
        key_word = dot['word']
        key_inx = dot['inx']
        x = dot['x']
        y = dot['y']
        #取成语
        word = searchCy(key_word,key_inx)
        #成语长度
        wordLen = len(word)
            
        if direct == 0: #横向
            #预备坐标点
            random_x = 0
            if abs(x)+wordLen >10:
                random_x = random.randint(abs(x)-4,abs(x))
            else:
                random_x = x + wordLen
            nextDots = [{'x':random_x,'y':y,'word':'','inx':0}]
            nextDirect = direct
            #判断坐标是否有冲突
            word_no_conflict = True
            only_two = True
            for i in range(wordLen):
                if str(x+i)+str(y) in res_dots:
                    if str(x+i)+"#"+str(y)+"#"+word[i:i+1] not in res_word_dots:
                        #查看是否有包含冲突字的成语
                        if only_two:
                            only_two = False
                            key_word2 = getKeyWord2(str(x+i)+"#"+str(y)+"#")
                            key_inx2 = i
                            word = searchCy2(key_word,key_inx,key_word2,key_inx2)
                            wordLen = len(word)
                            #重置循环位置
                            if word:
                                i = 0
                                word_no_conflict = True
                        else:
                            word_no_conflict = False
            
            if (abs(x) + wordLen) <= 10 \
                and (str(x-1)+str(y)) not in res_dots \
                and (str(x+wordLen)+str(y)) not in res_dots \
                and str(x)+str(y)+str(direct) not in start_dots \
                and word_no_conflict \
                and word:
                #加入字的坐标
                for i in range(wordLen):
                    res_dots.append(str(x+i)+str(y))
                    res_word_dots.append(str(x+i)+"#"+str(y)+"#"+word[i:i+1])
                
                #加入起点坐标
                start_dots.append(str(x)+str(y)+str(direct))
                
                #加入词
                res.append({'x':x,'y':y,'word':word,'direct':direct})
                
                print('=====横排加入符合条件的成语2: ',res)
                if len(res) < nums:
                    #以此成语，扩展其它成语的个数
                    nextCyNum = random.randint(1,int(wordLen/2))
                    nextInxs = getInxs(nextCyNum,wordLen)
                    #下一批成语的坐标
                    nextDots = []
                    for nextInx in nextInxs:
                        newX = x + nextInx
                        inx = random.randint(0,2)
                        newY = y + inx
                        nextDots.append({'x':newX,'y':newY,'word':word[nextInx:nextInx+1],'inx':inx})
                    #方向
                    nextDirect = 1
                
            #继续循环
            if len(res)+len(nextDots) <= nums:
                print('====横排下轮循环的参数3: ',nextDirect,nextDots)
                buildCy(nextDirect,nextDots)
        else: #纵向
            #预备坐标点
            random_y = 0
            if abs(y) - wordLen < 0:
                random_y = random.randint(abs(y)-4,abs(y))
            else:
                random_y = abs(y) - wordLen
            nextDots = [{'x':x,'y':random_y,'word':'','inx':0}]
            nextDirect = direct
            #判断坐标是否有冲突
            word_no_conflict = True
            only_two = True
            for i in range(wordLen):
                if (str(x)+str(y-i)) in res_dots:
                    if (str(x)+"#"+str(y-i)+"#"+word[i:i+1]) not in res_word_dots:
                        #查看是否有包含冲突字的成语
                        if only_two:
                            only_two = False
                            key_word2 = getKeyWord2(str(x)+"#"+str(y-i)+"#")
                            key_inx2 = i
                            word = searchCy2(key_word,key_inx,key_word2,key_inx2)
                            wordLen = len(word)
                            #重置循环位置
                            if word:
                                i = 0
                                word_no_conflict = True
                        else:
                            word_no_conflict = False
            
            if (abs(y) + wordLen) <= 10 \
                and (str(x)+str(y+1)) not in res_dots \
                and (str(x)+str(y-wordLen)) not in res_dots \
                and str(x)+str(y)+str(direct) not in start_dots \
                and word_no_conflict \
                and word:
                #加入字的坐标
                for i in range(wordLen):
                    res_dots.append(str(x)+str(y-i))
                    res_word_dots.append(str(x)+"#"+str(y-i)+"#"+word[i:i+1])
                    
                #加入起点坐标
                start_dots.append(str(x)+str(y)+str(direct))    
                    
                #加入词
                res.append({'x':x,'y':y,'word':word,'direct':direct})
                
                print('竖排加入成语4： ',res)
                if len(res) < nums:
                    #以此成语，扩展其它成语的个数
                    nextCyNum = random.randint(1,int(wordLen/2))
                    nextInxs = getInxs(nextCyNum,wordLen)
                    #下一批成语的坐标
                    nextDots = []
                    for nextInx in nextInxs:
                        inx = random.randint(0,2)
                        newX = x - inx
                        newY = y - nextInx
                        nextDots.append({'x':newX,'y':newY,'word':word[nextInx:nextInx+1],'inx':inx})
                    #方向
                    nextDirect = 0
            #继续循环
            if len(res)+len(nextDots) <= nums:
                print('竖排下轮参数5: ',nextDirect,nextDots)
                buildCy(nextDirect,nextDots)
                

In [10]:
#成语个数
cyNums = len(cys)

#当前关卡成语个数
lvl = 3
nums = getCyNum(lvl)

#方向，0，1
direct = random.randint(0,1)
#坐标
dots = [{'x':0,'y':0,'word':'','inx':0}]
#dots = [{'x':random.randint(0,9),'y':random.randint(0,9)}]
res = []

#坐标点
res_dots = []

#单个词的坐标点
res_word_dots = []

#起点坐标
start_dots = []

In [11]:
buildCy(direct,dots)

=====坐标起点1:  {'x': 0, 'y': 0, 'word': '', 'inx': 0}
竖排加入成语4：  [{'x': 0, 'y': 0, 'word': '投膏止火', 'direct': 1}]
竖排下轮参数5:  0 [{'x': 0, 'y': -2, 'word': '止', 'inx': 0}]
=====坐标起点1:  {'x': 0, 'y': -2, 'word': '止', 'inx': 0}
=====横排加入符合条件的成语2:  [{'x': 0, 'y': 0, 'word': '投膏止火', 'direct': 1}, {'x': 0, 'y': -2, 'word': '止戈兴仁', 'direct': 0}]
====横排下轮循环的参数3:  1 [{'x': 2, 'y': -1, 'word': '兴', 'inx': 1}]
=====坐标起点1:  {'x': 2, 'y': -1, 'word': '兴', 'inx': 1}
竖排加入成语4：  [{'x': 0, 'y': 0, 'word': '投膏止火', 'direct': 1}, {'x': 0, 'y': -2, 'word': '止戈兴仁', 'direct': 0}, {'x': 2, 'y': -1, 'word': '夙兴昧旦', 'direct': 1}]
竖排下轮参数5:  0 [{'x': 2, 'y': -4, 'word': '旦', 'inx': 0}]
=====坐标起点1:  {'x': 2, 'y': -4, 'word': '旦', 'inx': 0}
=====横排加入符合条件的成语2:  [{'x': 0, 'y': 0, 'word': '投膏止火', 'direct': 1}, {'x': 0, 'y': -2, 'word': '止戈兴仁', 'direct': 0}, {'x': 2, 'y': -1, 'word': '夙兴昧旦', 'direct': 1}, {'x': 2, 'y': -4, 'word': '旦暮入地', 'direct': 0}]
====横排下轮循环的参数3:  1 [{'x': 3, 'y': -3, 'word': '暮', 'inx': 1}, {'x': 5, 'y'

In [12]:
res

[{'x': 0, 'y': 0, 'word': '投膏止火', 'direct': 1},
 {'x': 0, 'y': -2, 'word': '止戈兴仁', 'direct': 0},
 {'x': 2, 'y': -1, 'word': '夙兴昧旦', 'direct': 1},
 {'x': 2, 'y': -4, 'word': '旦暮入地', 'direct': 0},
 {'x': 3, 'y': 3, 'word': '蝇头小楷', 'direct': 1},
 {'x': 2, 'y': 3, 'word': '青蝇吊客', 'direct': 0},
 {'x': 5, 'y': -2, 'word': '天冠地屦', 'direct': 1}]

In [13]:
# #手动调整位置坐标
# res.pop(5)
# 
# #调整字段坐标位置
# res_word_dots = []
# for cy_word in res:
#     x = int(cy_word['x'])
#     y = int(cy_word['y'])
#     cur_word = cy_word['word']
#     cur_word_len = len(cur_word)
#     cur_direct = int(cy_word['direct'])
#     if cur_direct == 0:
#         for i in range(cur_word_len):
#             res_word_dots.append(str(x+i)+"#"+str(y)+"#"+cur_word[i:i+1])
#     else:
#         for i in range(cur_word_len):
#             res_word_dots.append(str(x)+"#"+str(y-i)+"#"+cur_word[i:i+1])
        

In [14]:
res_word_dots

['0#0#投',
 '0#-1#膏',
 '0#-2#止',
 '0#-3#火',
 '0#-2#止',
 '1#-2#戈',
 '2#-2#兴',
 '3#-2#仁',
 '2#-1#夙',
 '2#-2#兴',
 '2#-3#昧',
 '2#-4#旦',
 '2#-4#旦',
 '3#-4#暮',
 '4#-4#入',
 '5#-4#地',
 '3#3#蝇',
 '3#2#头',
 '3#1#小',
 '3#0#楷',
 '2#3#青',
 '3#3#蝇',
 '4#3#吊',
 '5#3#客',
 '5#-2#天',
 '5#-3#冠',
 '5#-4#地',
 '5#-5#屦']

In [15]:
nums

6

In [16]:
res_df = pd.DataFrame(res_word_dots)
res_df[:5]

,0
0,0#0#投
1,0#-1#膏
2,0#-2#止
3,0#-3#火
4,0#-2#止


In [17]:
res_df['x'] = res_df[0].apply(lambda x: int(x.split("#")[0]))
res_df['y'] = res_df[0].apply(lambda x: int(x.split("#")[1]))
res_df['word'] = res_df[0].apply(lambda x:x.split("#")[2])
res_df = res_df.drop(columns=[0],axis=1)
res_df = res_df.drop_duplicates()
res_df[:5]

,x,y,word
0,0,0,投
1,0,-1,膏
2,0,-2,止
3,0,-3,火
5,1,-2,戈


In [18]:
# 缺失字
answers = []
for item in res:
    #空缺个数
    word = item['word']
    wordLen = len(word)
    nanNum = random.randint(1,wordLen-1)
    nanInxs = getInxs(nanNum,wordLen)
    for nanInx in nanInxs:
        answers.append(word[nanInx:nanInx+1])
        
answers

['投', '膏', '火', '戈', '兴', '兴', '旦', '地', '蝇', '吊', '冠']

In [19]:
x_min = res_df['x'].min()
x_max = res_df['x'].max()

y_min = res_df['y'].min()
y_max = res_df['y'].max()

x_gap = int((9 - abs(x_max - x_min))/2)
x_b = x_gap + 1 - x_min
x_dis = abs(x_max - x_min) + 1

y_gap = int((9 - abs(y_max - y_min))/2)
y_b = y_gap + 1 - y_min
y_dis = abs(y_max - y_min) + 1

sqls=[]

sql = "insert into llx.chengyu_level(lvl,row_code,word,answer) values(%d,'hang%d','%s','%s');"
if x_dis<=10 and y_dis<=10:
    sort_ress = res_df.sort_values(by=['y','x'],ascending=[1,1])
    
    row_num = 1
    start_inx = 1
    
    for inx,item in sort_ress.iterrows():
        real_x = item['x'] + x_b
        real_y = item['y'] + y_b
        real_word = item['word']
        
        print('===== x,y,word,row_num,start_inx:',real_x,real_y,real_word,row_num,start_inx)
        
        if row_num>1 and row_num < real_y:
            #加入空值
            while start_inx <= 10:
                tmp_sql = sql % (lvl,10 - row_num,'','')
                sqls.append(tmp_sql)
                start_inx += 1
            #重制位置
            start_inx = 1
        
        #加入空值
        while start_inx < real_x:
            tmp_sql = sql % (lvl,10 - real_y,'','')
            sqls.append(tmp_sql)
            start_inx += 1
        #加入当前值
        if real_word in answers:
            tmp_sql = sql % (lvl,10 - real_y,'',real_word)
        else:
            tmp_sql = sql % (lvl,10 - real_y,real_word,'')
        sqls.append(tmp_sql)
        
        row_num = real_y
        start_inx = real_x + 1
    
    #补充最后的空行
    while start_inx <= 10:
        tmp_sql = sql % (lvl,10 - row_num,'','')
        sqls.append(tmp_sql)
        start_inx += 1

===== x,y,word,row_num,start_inx: 8 1 屦 1 1
===== x,y,word,row_num,start_inx: 5 2 旦 1 9
===== x,y,word,row_num,start_inx: 6 2 暮 2 6
===== x,y,word,row_num,start_inx: 7 2 入 2 7
===== x,y,word,row_num,start_inx: 8 2 地 2 8
===== x,y,word,row_num,start_inx: 3 3 火 2 9
===== x,y,word,row_num,start_inx: 5 3 昧 3 4
===== x,y,word,row_num,start_inx: 8 3 冠 3 6
===== x,y,word,row_num,start_inx: 3 4 止 3 9
===== x,y,word,row_num,start_inx: 4 4 戈 4 4
===== x,y,word,row_num,start_inx: 5 4 兴 4 5
===== x,y,word,row_num,start_inx: 6 4 仁 4 6
===== x,y,word,row_num,start_inx: 8 4 天 4 7
===== x,y,word,row_num,start_inx: 3 5 膏 4 9
===== x,y,word,row_num,start_inx: 5 5 夙 5 4
===== x,y,word,row_num,start_inx: 3 6 投 5 6
===== x,y,word,row_num,start_inx: 6 6 楷 6 4
===== x,y,word,row_num,start_inx: 6 7 小 6 7
===== x,y,word,row_num,start_inx: 6 8 头 7 7
===== x,y,word,row_num,start_inx: 5 9 青 8 7
===== x,y,word,row_num,start_inx: 6 9 蝇 9 6
===== x,y,word,row_num,start_inx: 7 9 吊 9 7
===== x,y,word,row_num,start_inx

In [20]:
for sql in sqls:
    print(sql)

insert into llx.chengyu_level(lvl,row_code,word,answer) values(2,'hang9','','');
insert into llx.chengyu_level(lvl,row_code,word,answer) values(2,'hang9','','');
insert into llx.chengyu_level(lvl,row_code,word,answer) values(2,'hang9','','');
insert into llx.chengyu_level(lvl,row_code,word,answer) values(2,'hang9','','');
insert into llx.chengyu_level(lvl,row_code,word,answer) values(2,'hang9','','');
insert into llx.chengyu_level(lvl,row_code,word,answer) values(2,'hang9','','');
insert into llx.chengyu_level(lvl,row_code,word,answer) values(2,'hang9','','');
insert into llx.chengyu_level(lvl,row_code,word,answer) values(2,'hang9','屦','');
insert into llx.chengyu_level(lvl,row_code,word,answer) values(2,'hang8','','旦');
insert into llx.chengyu_level(lvl,row_code,word,answer) values(2,'hang8','暮','');
insert into llx.chengyu_level(lvl,row_code,word,answer) values(2,'hang8','入','');
insert into llx.chengyu_level(lvl,row_code,word,answer) values(2,'hang8','','地');
insert into llx.chengyu

In [21]:
# # 保存关卡结果
# with open('lvl_words.txt','a') as f:
#     for item in res:
#         f.write(str(lvl)+","+item['word']+'\n')


In [22]:
res_df['x'].min(),res_df['x'].max() 

(0, 5)

In [23]:
res_df['y'].min(),res_df['y'].max()

(-5, 3)

In [24]:
sort_ress = res_df.sort_values(by=['y','x'],ascending=[1,1])
sort_ress

,x,y,word
27,5,-5,屦
11,2,-4,旦
13,3,-4,暮
14,4,-4,入
15,5,-4,地
3,0,-3,火
10,2,-3,昧
25,5,-3,冠
2,0,-2,止
5,1,-2,戈


In [25]:
for inx,item in res_df.iterrows():
    print(inx,item['x'],item['y'],item['word'])
    break

0 0 0 投


In [26]:
black_sql = "insert into llx.chengyu_level(lvl,row_code,word,answer) values(1,'hang2','','');"
black_sql

"insert into llx.chengyu_level(lvl,row_code,word,answer) values(1,'hang2','','');"

In [27]:
sql = "insert into llx.chengyu_level(lvl,row_code,word,answer) values(%d,'hang%d','%s','%s');"
s = sql % (1,2,'a','')
s

"insert into llx.chengyu_level(lvl,row_code,word,answer) values(1,'hang2','a','');"

In [28]:
searchCy('天',2)

'浪迹天涯'